In [2]:
import sys
sys.path.append('../utils')

In [11]:
import pipeline
import importlib
import pandas as pd

importlib.reload(pipeline)
from pipeline import extract_session_data, clean_session_data, get_driver_laps, get_manual_finish_dict
from pipeline import assemble_race_dataset, extract_fp2_features, extract_quali_features, assemble_race_dataset_pre_race
from fastf1 import get_session

In [19]:
# Dutch GP 2025 (Round 15)
# This cell was made before the race so it excludes the race finish data 
df_fp2_15 = extract_session_data(2025, "Netherlands", "FP2")
df_quali_15 = extract_session_data(2025, "Netherlands", "Q")

full_driver_list = [
    'ALB', 'ALO', 'ANT', 'BEA', 'BOR', 'COL', 'GAS', 'HAD', 'HAM', 'HUL',
    'LAW', 'LEC', 'NOR', 'OCO', 'PIA', 'RUS', 'SAI', 'STR', 'TSU', 'VER'
]


fp2_clean_15 = clean_session_data(df_fp2_15)
quali_clean_15 = clean_session_data(df_quali_15)

fp2_features_15_raw = extract_fp2_features(fp2_clean_15)
fp2_features_15_raw = fp2_features_15_raw.drop_duplicates(subset="Driver", keep="first")

fp2_features_15 = pd.DataFrame({'Driver': full_driver_list})
fp2_features_15 = fp2_features_15.merge(fp2_features_15_raw, on='Driver', how='left')
# fp2_features_15['FP2_participated'] = fp2_features_15['fp2_avg_lap'].notna()

quali_features_15 = extract_quali_features(quali_clean_15)
                       
race_df_15 = fp2_features_15.merge(quali_features_15, on='Driver', how='outer')
# race_df_15["FP2_participated"] = race_df_15["fp2_avg_lap"].notna()
#race_df_15["Quali_participated"] = race_df_15["FastestQualiLap"].notna()
race_df_15["grand_prix"] = "Netherlands"
race_df_15

core           INFO 	Loading data for Dutch Grand Prix - Practice 2 [v3.4.4]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
logger      WARNING 	Failed to load result data from Ergast!
core        WARNING 	No result data for this session available on Ergast! (This is expected for recent sessions)
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/fastf1/core.py:1579: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '<TimedeltaArray>
['0 days 00:38:39.086000']
Length: 1, dtype: timedelta64[ns]' has dtype incompatible with dateti

,Driver,fp2_avg_lap,fp2_best_lap,fp2_total_laps,FastestQualiLap,QualiPosition,grand_prix
0,ALB,92.309167,71.756,6.0,69.652,15.0,Netherlands
1,ALO,87.701083,69.977,12.0,69.366,8.0,Netherlands
2,ANT,87.443125,71.185,8.0,69.493,11.0,Netherlands
3,BEA,77.386529,71.113,17.0,70.262,19.0,Netherlands
4,BOR,78.776938,71.320,16.0,69.622,12.0,Netherlands
5,COL,85.379154,70.957,13.0,70.104,16.0,Netherlands
6,GAS,83.918000,72.122,17.0,69.637,14.0,Netherlands
7,HAD,NaN,NaN,NaN,69.208,4.0,Netherlands
8,HAM,87.312538,70.738,13.0,69.261,6.0,Netherlands
9,HUL,79.281437,71.080,16.0,70.195,17.0,Netherlands


In [13]:
print(sorted(fp2_features_15_raw["Driver"].unique()))

['ALB', 'ALO', 'ANT', 'BEA', 'BOR', 'COL', 'GAS', 'HAM', 'HUL', 'LAW', 'LEC', 'NOR', 'OCO', 'PIA', 'RUS', 'SAI', 'STR', 'TSU', 'VER']
